In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

def crop_black_borders(image, threshold=10):
    """
    Crops out surrounding black or nearly black borders from the image.
    A pixel is considered non-black if any channel exceeds the threshold.
    """
    # Create a mask of where the image is not black
    # (using the maximum channel value per pixel)
    if len(image.shape) == 3:
        mask = image.max(axis=2) > threshold
    else:
        mask = image > threshold

    # Get coordinates of non-black pixels
    coords = np.argwhere(mask)
    if coords.size == 0:
        # No non-black pixels; return original image
        return image

    # Find the bounding box of the non-black region
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0) + 1  # slices are exclusive at the top
    cropped = image[y0:y1, x0:x1]
    return cropped

def compute_stitched_ssim(image_path, k, seam_width=10, black_thresh=10):
    """
    Computes the average SSIM for seam areas between adjacent sub-images
    in a K x K stitched image stored in 'image_path'.

    Parameters:
        image_path (str): The file path to the stitched image.
        k (int): The number of rows/columns in the stitched grid (total sub-images = k*k).
        seam_width (int): The width (in pixels) of the region along each seam to compare.
        black_thresh (int): Pixel intensity threshold to consider as black when cropping.

    Returns:
        average_ssim (float): The average SSIM computed over all seams.
        seam_ssim_scores (dict): A dictionary with keys 'vertical' and 'horizontal' containing lists of SSIM scores.
    """
    # Load the image using OpenCV (reads as BGR)
    image_bgr = cv2.imread(image_path)
    if image_bgr is None:
        raise FileNotFoundError(f"Could not load image from '{image_path}'")

    # Convert to RGB for consistency
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # Crop out surrounding black borders
    cropped_image = crop_black_borders(image_rgb, threshold=black_thresh)

    # Get dimensions of the cropped image
    h, w = cropped_image.shape[:2]

    # Determine size of each sub-image (using integer division)
    sub_h = h // k
    sub_w = w // k

    # Create a list to hold the sub-images
    sub_images = []
    for i in range(k):
        row = []
        for j in range(k):
            # Extract the (i, j)th sub-image
            x_start = j * sub_w
            x_end = (j + 1) * sub_w
            y_start = i * sub_h
            y_end = (i + 1) * sub_h
            sub_img = cropped_image[y_start:y_end, x_start:x_end]
            row.append(sub_img)
        sub_images.append(row)

    # Prepare lists to hold SSIM scores from vertical and horizontal seams
    vertical_scores = []
    horizontal_scores = []

    # Compute SSIM on vertical seams (between left & right adjacent sub-images)
    for i in range(k):
        for j in range(k - 1):
            left_img = sub_images[i][j]
            right_img = sub_images[i][j + 1]
            # Select the right seam of left_img and left seam of right_img
            left_seam = left_img[:, -seam_width:]
            right_seam = right_img[:, :seam_width]
            # Convert to grayscale for SSIM calculation
            left_gray = cv2.cvtColor(left_seam, cv2.COLOR_RGB2GRAY)
            right_gray = cv2.cvtColor(right_seam, cv2.COLOR_RGB2GRAY)
            score, _ = ssim(left_gray, right_gray, full=True)
            vertical_scores.append(score)

    # Compute SSIM on horizontal seams (between top & bottom adjacent sub-images)
    for i in range(k - 1):
        for j in range(k):
            top_img = sub_images[i][j]
            bottom_img = sub_images[i + 1][j]
            # Select the bottom seam of top_img and top seam of bottom_img
            top_seam = top_img[-seam_width:, :]
            bottom_seam = bottom_img[:seam_width, :]
            top_gray = cv2.cvtColor(top_seam, cv2.COLOR_RGB2GRAY)
            bottom_gray = cv2.cvtColor(bottom_seam, cv2.COLOR_RGB2GRAY)
            score, _ = ssim(top_gray, bottom_gray, full=True)
            horizontal_scores.append(score)

    # Combine all scores and compute an average SSIM
    all_scores = vertical_scores + horizontal_scores
    # average_ssim = np.mean(all_scores) if all_scores else None
    penalized_average_ssim = 1 - np.mean((np.array(all_scores) - 0.6)**2)

    seam_ssim_scores = {
        "vertical": vertical_scores,
        "horizontal": horizontal_scores
    }

    return penalized_average_ssim, seam_ssim_scores

# Example usage:
# if __name__ == "__main__":
#     # Specify the grid size (K value)
#     k = 5  # for a 3x3 stitching, for example
#     try:

#         print("kaze_194_1.0.jpg")
#         avg_ssim, scores = compute_stitched_ssim("kaze_194_1.0.jpg", k, seam_width=50, black_thresh=10)
#         print(f"Average SSIM for seams: {avg_ssim:.4f}")
#         print("Vertical seam scores:", scores["vertical"])
#         print("Horizontal seam scores:", scores["horizontal"])

#         print("\nsuperpoint_194_0.25.jpg")
#         avg_ssim, scores = compute_stitched_ssim("superpoint_194_0.25.jpg", k, seam_width=50, black_thresh=10)
#         print(f"Average SSIM for seams: {avg_ssim:.4f}")
#         print("Vertical seam scores:", scores["vertical"])
#         print("Horizontal seam scores:", scores["horizontal"])

#         print("\nsuperpoint_194_1.0.jpg")
#         avg_ssim, scores = compute_stitched_ssim("superpoint_194_1.0.jpg", k, seam_width=50, black_thresh=10)
#         print(f"Average SSIM for seams: {avg_ssim:.4f}")
#         print("Vertical seam scores:", scores["vertical"])
#         print("Horizontal seam scores:", scores["horizontal"])

#         print("\n\n")
#         print("kaze_234_0.25.jpg")
#         avg_ssim, scores = compute_stitched_ssim("kaze_234_0.25.jpg", k, seam_width=50, black_thresh=10)
#         print(f"Average SSIM for seams: {avg_ssim:.4f}")
#         print("Vertical seam scores:", scores["vertical"])
#         print("Horizontal seam scores:", scores["horizontal"])

#         print("\nkaze_234_0.75.jpg")
#         avg_ssim, scores = compute_stitched_ssim("kaze_234_0.75.jpg", k, seam_width=50, black_thresh=10)
#         print(f"Average SSIM for seams: {avg_ssim:.4f}")
#         print("Vertical seam scores:", scores["vertical"])
#         print("Horizontal seam scores:", scores["horizontal"])

#         print("\nkaze_234_0.75.jpg")
#         avg_ssim, scores = compute_stitched_ssim("kaze_234_0.75.jpg", k, seam_width=50, black_thresh=10)
#         print(f"Average SSIM for seams: {avg_ssim:.4f}")
#         print("Vertical seam scores:", scores["vertical"])
#         print("Horizontal seam scores:", scores["horizontal"])

#         print("\nkaze_234_1.0.jpg")
#         avg_ssim, scores = compute_stitched_ssim("kaze_234_1.0.jpg", k, seam_width=50, black_thresh=10)
#         print(f"Average SSIM for seams: {avg_ssim:.4f}")
#         print("Vertical seam scores:", scores["vertical"])
#         print("Horizontal seam scores:", scores["horizontal"])

#         print("\nkaze_234_1.0_offset_1.jpg")
#         avg_ssim, scores = compute_stitched_ssim("kaze_234_1.0_offset_1.jpg", k, seam_width=50, black_thresh=10)
#         print(f"Average SSIM for seams: {avg_ssim:.4f}")
#         print("Vertical seam scores:", scores["vertical"])
#         print("Horizontal seam scores:", scores["horizontal"])

#         print("\n\nsuperpoint_234_1.0.jpg")
#         avg_ssim, scores = compute_stitched_ssim("superpoint_234_1.0.jpg", k, seam_width=50, black_thresh=10)
#         print(f"Average SSIM for seams: {avg_ssim:.4f}")
#         print("Vertical seam scores:", scores["vertical"])
#         print("Horizontal seam scores:", scores["horizontal"])

#         print("\nsuperpoint_234_1.0_offset_1.jpg")
#         avg_ssim, scores = compute_stitched_ssim("superpoint_234_1.0_offset_1.jpg", k, seam_width=50, black_thresh=10)
#         print(f"Average SSIM for seams: {avg_ssim:.4f}")
#         print("Vertical seam scores:", scores["vertical"])
#         print("Horizontal seam scores:", scores["horizontal"])


#     except Exception as e:
#         print("Error:", e)

In [ ]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 18.1 MB/s eta 0:00:00


In [ ]:
print("\nsift_194_1.0.png")
avg_ssim, scores = compute_stitched_ssim("sift_234_1.0.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\nsift_234_1.0_offset_1.jpg")
avg_ssim, scores = compute_stitched_ssim("sift_234_1.0_offset_1.jpg", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

# print("\nsift_234_1.0_offset_1_v2.jpg")
# avg_ssim, scores = compute_stitched_ssim("sift_234_1.0_offset_1_v2.jpg", 5, seam_width=10, black_thresh=10)
# print(f"Average SSIM for seams: {avg_ssim:.4f}")
# print("Vertical seam scores:", scores["vertical"])
# print("Horizontal seam scores:", scores["horizontal"])


sift_234_1.0.png
Average SSIM for seams: 0.9801
Vertical seam scores: [np.float64(0.5474037623511475), np.float64(0.44922066180276304), np.float64(0.5979073785408155), np.float64(0.5879883963831364), np.float64(0.7619817996049992), np.float64(0.5238036949035108), np.float64(0.688483390427944), np.float64(0.6202652660594088), np.float64(0.6761504343112629), np.float64(0.5623821820069502), np.float64(0.6309831361554015), np.float64(0.3631382202059126), np.float64(0.6171211276195241), np.float64(0.4402018663118193), np.float64(0.45913389609626143), np.float64(0.3678725935318063), np.float64(0.6913794010383829), np.float64(0.5006488142240515), np.float64(0.46133951138625706), np.float64(0.33753581956794915)]
Horizontal seam scores: [np.float64(0.5676543541993895), np.float64(0.5558296702316314), np.float64(0.7127325045795296), np.float64(0.6042204072743335), np.float64(0.8610211080956425), np.float64(0.6241088016576156), np.float64(0.47976685722975204), np.float64(0.5184870041000741), np.

In [ ]:
print("\n5_KAZE.png")
avg_ssim, scores = compute_stitched_ssim("5_KAZE.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n5_SIFT.png")
avg_ssim, scores = compute_stitched_ssim("5_SIFT.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n5_SuperPoint.png")
avg_ssim, scores = compute_stitched_ssim("5_SuperPoint.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


5_KAZE.png
Average SSIM for seams: 0.8887
Vertical seam scores: [np.float64(0.2906251547604395), np.float64(0.3558387346546615), np.float64(0.27950437558763125), np.float64(0.21602238289527506), np.float64(0.3296209958139622), np.float64(0.23058379782133093), np.float64(0.2780085004980369), np.float64(0.22076463228295684), np.float64(0.2048625313889261), np.float64(0.26777964053751235), np.float64(0.2319524045262845), np.float64(0.2392948982487754), np.float64(0.3120562472838032), np.float64(0.3408537886019003), np.float64(0.12931832224014891), np.float64(0.427884260627665), np.float64(0.3669665299195931), np.float64(0.4364887060580928), np.float64(0.3765531993921648), np.float64(0.5662884043353612)]
Horizontal seam scores: [np.float64(0.3050711940288216), np.float64(0.30763562031737285), np.float64(0.23657915053727502), np.float64(0.28833406559532776), np.float64(0.26521929056153537), np.float64(0.26505049287120935), np.float64(0.16965746385059702), np.float64(0.26227487708647457), n

In [ ]:
print("\n6_SuperPoint.jpg")
avg_ssim, scores = compute_stitched_ssim("6_SuperPoint.jpg", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n6_SuperPoint.png")
avg_ssim, scores = compute_stitched_ssim("6_SuperPoint.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n6_SIFT.jpg")
avg_ssim, scores = compute_stitched_ssim("6_SIFT.jpg", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n6_SIFT.png")
avg_ssim, scores = compute_stitched_ssim("6_SIFT.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


6_SuperPoint.jpg
Average SSIM for seams: 0.8460
Vertical seam scores: [np.float64(0.45336096415623367), np.float64(0.6548422992064428), np.float64(0.8642134570512441), np.float64(1.0), np.float64(0.22750466190718005), np.float64(0.23063581718465015), np.float64(0.10767432029856366), np.float64(0.22998221807278807), np.float64(0.14618150497731636), np.float64(0.17829103153216788), np.float64(0.21242625651912259), np.float64(0.18968106147493344), np.float64(0.3497368222501672), np.float64(0.307190447624186), np.float64(0.12389846456542089), np.float64(0.0805756024942528), np.float64(0.34316318098063564), np.float64(0.09919903562187395), np.float64(0.10961179341257099), np.float64(0.08691230473352153)]
Horizontal seam scores: [np.float64(0.3399662820060265), np.float64(0.31547368427682704), np.float64(0.20417326756529675), np.float64(0.18530319202872095), np.float64(1.0), np.float64(0.25584257749748635), np.float64(0.08258759625657426), np.float64(0.14023329406319476), np.float64(0.14997

In [ ]:
print("\n1_SIFT.png")
avg_ssim, scores = compute_stitched_ssim("1_SIFT.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n1_SIFT_v2.png")
avg_ssim, scores = compute_stitched_ssim("1_SIFT_v2.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n1_SIFT_v3.png")
avg_ssim, scores = compute_stitched_ssim("1_SIFT_v3.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n1_SuperPoint.png")
avg_ssim, scores = compute_stitched_ssim("1_SuperPoint.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n1_SuperPoint_v2.png")
avg_ssim, scores = compute_stitched_ssim("1_SuperPoint_v2.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


1_SIFT.png
Average SSIM for seams: 0.9865
Vertical seam scores: [np.float64(0.6041633300871082), np.float64(0.8430759653706323), np.float64(0.7242607347873817), np.float64(0.7292748872727399), np.float64(0.462922353121223), np.float64(0.7311847509080193), np.float64(0.6981898765105444), np.float64(0.7126327905772877), np.float64(0.7146460527003144), np.float64(0.4153422732171962), np.float64(0.7639444134762992), np.float64(0.6095639773081237), np.float64(0.4310420451107533), np.float64(0.4024087134249543), np.float64(0.5840927276645548), np.float64(0.5759598182282925), np.float64(0.6814554241564593), np.float64(0.7499473148740275), np.float64(0.7962085559723826), np.float64(0.7182514398797983)]
Horizontal seam scores: [np.float64(0.6111207534358886), np.float64(0.6785671433910945), np.float64(0.7400751401014436), np.float64(0.6585314312834256), np.float64(0.7057252641696106), np.float64(0.6907916415981126), np.float64(0.48881728359484483), np.float64(0.58847856544152), np.float64(0.61

In [ ]:
print("\n2_SIFT.png")
avg_ssim, scores = compute_stitched_ssim("2_SIFT.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n2_SuperPoint.png")
avg_ssim, scores = compute_stitched_ssim("2_SuperPoint.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


2_SIFT.png
Average SSIM for seams: 0.8633
Vertical seam scores: [np.float64(0.35256300007509944), np.float64(0.3620404854818079), np.float64(0.32541605494196474), np.float64(0.33679617610019086), np.float64(0.306426397513574), np.float64(0.1923631966162545), np.float64(0.18035446903066907), np.float64(0.1495007470895925), np.float64(0.1674418435753472), np.float64(0.2269659917126749), np.float64(0.1325437526839757), np.float64(0.15690974670373242), np.float64(0.24538472857667448), np.float64(0.17901793294765234), np.float64(0.11433489738394241), np.float64(0.12708057320911945), np.float64(0.2671293670936841), np.float64(0.4069903155628418), np.float64(0.6148267119984415), np.float64(0.3467689760384923)]
Horizontal seam scores: [np.float64(0.3378426985448331), np.float64(0.34703400378858984), np.float64(0.24997197203937238), np.float64(0.24011691649746256), np.float64(0.27651935324806526), np.float64(0.284013327954578), np.float64(0.3448475998262319), np.float64(0.1983467109085736), np

In [ ]:
print("\n3_SIFT.png")
avg_ssim, scores = compute_stitched_ssim("3_SIFT.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n3_SIFT_v2.png")
avg_ssim, scores = compute_stitched_ssim("3_SIFT_v2.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n3_SIFT_v3.png")
avg_ssim, scores = compute_stitched_ssim("3_SIFT_v3.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n3_SuperPoint.png")
avg_ssim, scores = compute_stitched_ssim("3_SuperPoint.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n3_SuperPoint_v2.png")
avg_ssim, scores = compute_stitched_ssim("3_SuperPoint_v2.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


3_SIFT.png
Average SSIM for seams: 0.9252
Vertical seam scores: [np.float64(0.40094672496849704), np.float64(0.4218383897381461), np.float64(0.3638976940710095), np.float64(0.19080122936701346), np.float64(0.35789089343503727), np.float64(0.386134133870133), np.float64(0.2876671976024058), np.float64(0.4139725977778532), np.float64(0.4639840264137748), np.float64(0.32707584876303464), np.float64(0.38687720116507224), np.float64(0.2890440415959051), np.float64(0.4567387435277933), np.float64(0.31181447421745184), np.float64(0.3133478292752611), np.float64(0.24974697180719604), np.float64(0.45359988460583023), np.float64(0.28365602357325453), np.float64(0.29653992117408684), np.float64(0.27591227939663876)]
Horizontal seam scores: [np.float64(0.5579739978338822), np.float64(0.4625764860109559), np.float64(0.27935724766274095), np.float64(0.14495515855388427), np.float64(0.997736134537001), np.float64(0.5886067628759102), np.float64(0.38590685430581834), np.float64(0.4069315937787196), n

In [ ]:
print("\n9_SuperPoint.png")
avg_ssim, scores = compute_stitched_ssim("9_SuperPoint.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n9_SIFT.png")
avg_ssim, scores = compute_stitched_ssim("9_SIFT.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n9_KAZE_v2.png")
avg_ssim, scores = compute_stitched_ssim("9_KAZE_v2.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n9_KAZE.png")
avg_ssim, scores = compute_stitched_ssim("9_KAZE.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


9_SuperPoint.png
Average SSIM for seams: 0.9200
Vertical seam scores: [np.float64(0.379149431701993), np.float64(0.33185349894290683), np.float64(0.39540417007931233), np.float64(0.5040140448084999), np.float64(0.31483052931059957), np.float64(0.39002694009484934), np.float64(0.4315903220912803), np.float64(0.45634850738599664), np.float64(0.338407076304237), np.float64(0.3677436994364623), np.float64(0.37831333843899184), np.float64(0.35319756050552487), np.float64(0.36766159493317757), np.float64(0.37221174990050143), np.float64(0.16321789726657232), np.float64(0.2650178589460505), np.float64(0.23503436362070554), np.float64(0.3223945340818014), np.float64(0.1881807675413033), np.float64(0.0895316718401416)]
Horizontal seam scores: [np.float64(0.4332229608126775), np.float64(0.4373197763208273), np.float64(0.3958591168466541), np.float64(0.5415985853159504), np.float64(0.44451164949497846), np.float64(0.3243942175682851), np.float64(0.3955485321077173), np.float64(0.28662836022532),

In [ ]:
print("\n7_KAZE.png")
avg_ssim, scores = compute_stitched_ssim("7_KAZE.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n7_SIFT.png")
avg_ssim, scores = compute_stitched_ssim("7_SIFT.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n7_SuperPoint.png")
avg_ssim, scores = compute_stitched_ssim("7_SuperPoint.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


7_KAZE.png
Average SSIM for seams: 0.9342
Vertical seam scores: [np.float64(0.4151042525684361), np.float64(0.514455458267715), np.float64(0.472908762555895), np.float64(0.40310011414819846), np.float64(0.43509393531133717), np.float64(0.4113875633477944), np.float64(0.29584081479143587), np.float64(0.4178199820030609), np.float64(0.2616861966801829), np.float64(0.37867557419024084), np.float64(0.4090744148966811), np.float64(0.3804309212660328), np.float64(0.22452210999634734), np.float64(0.2337626026097134), np.float64(0.21390880422779843), np.float64(0.19842411971455445), np.float64(0.3229424707578141), np.float64(0.2413435396136779), np.float64(0.25395276251760784), np.float64(0.26148242406167477)]
Horizontal seam scores: [np.float64(0.4354115165801818), np.float64(0.5756465885616681), np.float64(0.581296516500802), np.float64(0.39098366314271743), np.float64(0.3196666767400806), np.float64(0.44296422609397557), np.float64(0.4776264903291366), np.float64(0.4680399079998978), np.fl

In [ ]:
print("\n8_SIFT.png")
avg_ssim, scores = compute_stitched_ssim("8_SIFT.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n8_SuperPoint.png")
avg_ssim, scores = compute_stitched_ssim("8_SuperPoint.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


8_SIFT.png
Average SSIM for seams: 0.9056
Vertical seam scores: [np.float64(0.744978935793814), np.float64(0.4749405466249346), np.float64(0.23116062551957442), np.float64(0.41114553819864397), np.float64(0.22049431402520217), np.float64(0.33935952795475727), np.float64(0.37629463663767954), np.float64(0.354076615407108), np.float64(0.1189778969455004), np.float64(0.2179277441989551), np.float64(0.3098419351767168), np.float64(0.25684158563840365), np.float64(0.20154186001299124), np.float64(0.4506074534054264), np.float64(0.3338628408586347), np.float64(0.25918300974645586), np.float64(0.454515486459864), np.float64(0.23743450662127286), np.float64(0.2057861925588285), np.float64(0.1533648469339116)]
Horizontal seam scores: [np.float64(0.7186436151873951), np.float64(0.39181970633852803), np.float64(0.3058923118399652), np.float64(0.26492403636972733), np.float64(0.34464984441004365), np.float64(0.39654442466129747), np.float64(0.23379683219044975), np.float64(0.45826751817624267), n

In [ ]:
print("\n11_KAZE.png")
avg_ssim, scores = compute_stitched_ssim("11_KAZE.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n11_SIFT.png")
avg_ssim, scores = compute_stitched_ssim("11_SIFT.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n11_SuperPoint.png")
avg_ssim, scores = compute_stitched_ssim("11_SuperPoint.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


11_KAZE.png
Average SSIM for seams: 0.9516
Vertical seam scores: [np.float64(0.5044462887587593), np.float64(0.534621441768334), np.float64(0.3390592823789133), np.float64(0.5330582747363076), np.float64(0.3911162999757611), np.float64(0.3926418777814686), np.float64(0.21211559517444023), np.float64(0.25047700841957227), np.float64(0.3599081920509544), np.float64(0.3935693578298994), np.float64(0.2667146159844739), np.float64(0.4251897172104976), np.float64(0.4181947261428152), np.float64(0.35276435716169785), np.float64(0.3728385670979978), np.float64(0.4318054414360795), np.float64(0.5085227902980414), np.float64(0.42065748403905207), np.float64(0.24750090662265908), np.float64(0.3171499013664257)]
Horizontal seam scores: [np.float64(0.3866995304257591), np.float64(0.5437803747742173), np.float64(0.4954502387447692), np.float64(0.4701173103037913), np.float64(0.7646613431945124), np.float64(0.4171047225707877), np.float64(0.3979737726213088), np.float64(0.3410968922181022), np.float

In [ ]:
print("\n12_KAZE.png")
avg_ssim, scores = compute_stitched_ssim("12_KAZE.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n12_SIFT.png")
avg_ssim, scores = compute_stitched_ssim("12_SIFT.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n12_SuperPoint.png")
avg_ssim, scores = compute_stitched_ssim("12_SuperPoint.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


12_KAZE.png
Average SSIM for seams: 0.8586
Vertical seam scores: [np.float64(0.30059874303257506), np.float64(0.4645649618358224), np.float64(0.30832924668778017), np.float64(0.2788308610107412), np.float64(0.2016873842877178), np.float64(0.2709355768589615), np.float64(0.21767719452244155), np.float64(0.17947545656638111), np.float64(0.3338044954645895), np.float64(0.1781247137576036), np.float64(0.09677059444360053), np.float64(0.11339065930200788), np.float64(0.28436612018720875), np.float64(0.23521740670005897), np.float64(0.15049998696937722), np.float64(0.1375284741662375), np.float64(0.41045436354349885), np.float64(0.5238865104031565), np.float64(0.5230339227041206), np.float64(0.5218781530209536)]
Horizontal seam scores: [np.float64(0.24171729526799265), np.float64(0.2098485684745403), np.float64(0.3143398805434889), np.float64(0.1726369533265693), np.float64(0.1213623112850493), np.float64(0.2096763440885005), np.float64(0.2251855007803008), np.float64(0.20138807684600019), 

In [ ]:
print("\n31_KAZE.png")
avg_ssim, scores = compute_stitched_ssim("31_KAZE.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

# print("\n31_SIFT.png")
# avg_ssim, scores = compute_stitched_ssim("31_SIFT.png", 5, seam_width=50, black_thresh=10)
# print(f"Average SSIM for seams: {avg_ssim:.4f}")
# print("Vertical seam scores:", scores["vertical"])
# print("Horizontal seam scores:", scores["horizontal"])

print("\n31_SuperPoint.png")
avg_ssim, scores = compute_stitched_ssim("31_SuperPoint.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


31_KAZE.png
Average SSIM for seams: 0.9795
Vertical seam scores: [np.float64(0.8118151440459004), np.float64(0.5321583076307068), np.float64(0.5220003503976336), np.float64(0.5457041769838036), np.float64(0.40910533156910894), np.float64(0.7544899946484154), np.float64(0.6898015291148664), np.float64(0.48055732299486253), np.float64(0.7658890576116361), np.float64(0.6135912949599273), np.float64(0.701125628545877), np.float64(0.5880467453142108), np.float64(0.509610989456951), np.float64(0.4343223341195038), np.float64(0.49094091192561645), np.float64(0.5674274084650713), np.float64(0.6191095726972176), np.float64(0.673622331708583), np.float64(0.8167729216469228), np.float64(0.6553002660775541)]
Horizontal seam scores: [np.float64(0.6918287578060093), np.float64(0.4859878877158099), np.float64(0.46840364418816166), np.float64(0.6841901618214782), np.float64(0.8338075203654003), np.float64(0.7333005777974655), np.float64(0.4249520640596675), np.float64(0.5719781708254575), np.float64(

In [ ]:
print("\n33_KAZE.png")
avg_ssim, scores = compute_stitched_ssim("33_KAZE.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n33_SIFT.png")
avg_ssim, scores = compute_stitched_ssim("33_SIFT.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

# print("\n33_SuperPoint.png")
# avg_ssim, scores = compute_stitched_ssim("31_SuperPoint.png", 5, seam_width=50, black_thresh=10)
# print(f"Average SSIM for seams: {avg_ssim:.4f}")
# print("Vertical seam scores:", scores["vertical"])
# print("Horizontal seam scores:", scores["horizontal"])


33_KAZE.png
Average SSIM for seams: 0.9644
Vertical seam scores: [np.float64(0.5886542587203586), np.float64(0.3999615800668204), np.float64(0.44681435385535645), np.float64(0.3822642236529063), np.float64(0.44105723864138086), np.float64(0.4897934467733326), np.float64(0.4462930059359716), np.float64(0.4710193430968507), np.float64(0.5100919892840297), np.float64(0.4037278603947591), np.float64(0.3670509649821357), np.float64(0.33085763126497936), np.float64(0.4479209681876842), np.float64(0.5199351478678551), np.float64(0.4342859234535197), np.float64(0.23779010360069275), np.float64(0.6303374760272855), np.float64(0.7572193296950198), np.float64(0.6482763697234528), np.float64(0.3989307345815684)]
Horizontal seam scores: [np.float64(0.43548280853895965), np.float64(0.4040987923011247), np.float64(0.4543943922358326), np.float64(0.5189220406094228), np.float64(0.45379228118425524), np.float64(0.3679028105313831), np.float64(0.5713879984199037), np.float64(0.4343861606913745), np.flo

In [ ]:
print("\n36_KAZE.png")
avg_ssim, scores = compute_stitched_ssim("36_KAZE.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n36_SuperPoint.png")
avg_ssim, scores = compute_stitched_ssim("36_SuperPoint.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n36_SIFT.png")
avg_ssim, scores = compute_stitched_ssim("36_SIFT.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


36_KAZE.png
Average SSIM for seams: 0.9420
Vertical seam scores: [np.float64(0.47990627760796833), np.float64(0.4822959701029684), np.float64(0.30434927345146895), np.float64(0.37776385379337424), np.float64(0.5729585513684379), np.float64(0.3552104496138079), np.float64(0.40670103553118014), np.float64(0.3507514783893366), np.float64(0.39617080421657724), np.float64(0.2986831619535559), np.float64(0.36699779572502955), np.float64(0.31357560291203085), np.float64(0.4836936457577596), np.float64(0.29768447654403485), np.float64(0.3608474876956113), np.float64(0.17206786554429085), np.float64(0.32981512107354566), np.float64(0.4755570833813409), np.float64(0.5138316822930189), np.float64(0.543312546754333)]
Horizontal seam scores: [np.float64(0.5323373483398589), np.float64(0.5563997760885826), np.float64(0.2885480262060284), np.float64(0.26850854394753565), np.float64(0.2698220593782463), np.float64(0.5053453563555125), np.float64(0.41499558835520606), np.float64(0.30083005094565), np.

In [ ]:
print("\n49_KAZE.jpg")
avg_ssim, scores = compute_stitched_ssim("49_KAZE.jpg", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n49_KAZE.png")
avg_ssim, scores = compute_stitched_ssim("49_KAZE.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n49_SuperPoint.jpg")
avg_ssim, scores = compute_stitched_ssim("49_SuperPoint.jpg", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n49_SuperPoint.png")
avg_ssim, scores = compute_stitched_ssim("49_SuperPoint.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n49_SIFT.jpg")
avg_ssim, scores = compute_stitched_ssim("49_SIFT.jpg", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n49_SIFT.png")
avg_ssim, scores = compute_stitched_ssim("49_SIFT.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


49_KAZE.jpg
Average SSIM for seams: 0.9453
Vertical seam scores: [np.float64(0.563789025869395), np.float64(0.48777152495013326), np.float64(0.41219738012702856), np.float64(0.5956531600878227), np.float64(0.47927856099001187), np.float64(0.44380065443668726), np.float64(0.4021062964004122), np.float64(0.5250487419854973), np.float64(0.2613009766364941), np.float64(0.3777346209377767), np.float64(0.36716843588855014), np.float64(0.7055628177619787), np.float64(0.364013521783989), np.float64(0.3190890297200651), np.float64(0.8966274067743534), np.float64(1.0), np.float64(0.4794985109812566), np.float64(0.3775246286388011), np.float64(1.0), np.float64(1.0)]
Horizontal seam scores: [np.float64(0.5625014006109574), np.float64(0.5519288377212136), np.float64(0.3954010598857583), np.float64(0.45727293281932907), np.float64(0.9285214789916191), np.float64(0.49385780360872195), np.float64(0.4986723128133026), np.float64(0.4120959680217796), np.float64(0.38855501901375666), np.float64(0.631342

In [ ]:
print("\n53_KAZE.jpg")
avg_ssim, scores = compute_stitched_ssim("53_KAZE.jpg", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n53_KAZE.png")
avg_ssim, scores = compute_stitched_ssim("53_KAZE.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n53_SuperPoint.jpg")
avg_ssim, scores = compute_stitched_ssim("53_SuperPoint.jpg", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n53_SuperPoint.png")
avg_ssim, scores = compute_stitched_ssim("53_SuperPoint.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n53_SIFT.jpg")
avg_ssim, scores = compute_stitched_ssim("53_SIFT.jpg", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n53_SIFT.png")
avg_ssim, scores = compute_stitched_ssim("53_SIFT.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


53_KAZE.jpg
Average SSIM for seams: 0.8912
Vertical seam scores: [np.float64(0.891074462994848), np.float64(0.6853336586509704), np.float64(0.49958031321147056), np.float64(0.31856296960196895), np.float64(0.2633358820409691), np.float64(0.2567150133574693), np.float64(0.3412020864923998), np.float64(0.21885017926894376), np.float64(0.28687052103428184), np.float64(0.22854830268036033), np.float64(0.18492822103447287), np.float64(0.1428912501620685), np.float64(0.2843736614701738), np.float64(0.12887266945127698), np.float64(0.1820945821377148), np.float64(0.17398288063582515), np.float64(0.24328702281471584), np.float64(0.23087915774044285), np.float64(0.276461000252468), np.float64(0.2601951866572589)]
Horizontal seam scores: [np.float64(0.7955495662312453), np.float64(0.34113932020971033), np.float64(0.37618476824726027), np.float64(0.2672499004487908), np.float64(0.44943489437002404), np.float64(0.37244035692391236), np.float64(0.28520555732611913), np.float64(0.32480063985030533)

In [ ]:
print("\n026_SIFT.jpg")
avg_ssim, scores = compute_stitched_ssim("026_SIFT.jpg", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


026_SIFT.jpg
Average SSIM for seams: 0.9547
Vertical seam scores: [np.float64(0.781516355298922), np.float64(0.8457857144937945), np.float64(0.8267685896632744), np.float64(0.7666688907819125), np.float64(0.7959109388813136), np.float64(0.7087815851158011), np.float64(0.9125736820973099), np.float64(0.671132477588565), np.float64(0.5536074710624334), np.float64(0.6764633479474295), np.float64(0.6296923586077169), np.float64(0.7998810328673989), np.float64(0.9418313610777638), np.float64(0.6588596961632641), np.float64(0.7701221522224615), np.float64(0.6778649406353273), np.float64(0.7780471448543853), np.float64(0.7677199511097479), np.float64(1.0), np.float64(0.9921588316050531)]
Horizontal seam scores: [np.float64(0.7675712201821622), np.float64(0.5708465117755026), np.float64(0.719102241230308), np.float64(0.6239046725102564), np.float64(0.8572697788132485), np.float64(0.7671001164160484), np.float64(0.7758723272863184), np.float64(0.8046243970940884), np.float64(0.952169918508357)

In [ ]:
print("\n051_KAZE.png")
avg_ssim, scores = compute_stitched_ssim("051_KAZE.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n051_SIFT.jpg")
avg_ssim, scores = compute_stitched_ssim("051_SIFT.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


051_KAZE.png
Average SSIM for seams: 0.9350
Vertical seam scores: [np.float64(0.4034994881012559), np.float64(0.5894914586701104), np.float64(0.27401450297647384), np.float64(0.35934921563342476), np.float64(0.513109333033237), np.float64(0.42449013348599984), np.float64(0.1673174547559802), np.float64(0.325935825274096), np.float64(0.5875445802304068), np.float64(0.4621117496012558), np.float64(0.5431774565044148), np.float64(0.23225937989359438), np.float64(0.2386413734308408), np.float64(0.4909014914630465), np.float64(0.3989551548700785), np.float64(0.1496473319425819), np.float64(0.5650367552681202), np.float64(0.7535821974709194), np.float64(0.9955706234927996), np.float64(0.8465189521828432)]
Horizontal seam scores: [np.float64(0.31141729032257676), np.float64(0.36060676049855644), np.float64(0.2554206536463269), np.float64(0.30329853323585493), np.float64(0.23640225528916836), np.float64(0.5228257646671036), np.float64(0.4181853822507516), np.float64(0.49571971110216967), np.f

In [ ]:
print("\n156_KAZE.png")
avg_ssim, scores = compute_stitched_ssim("156_KAZE.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])

print("\n156_SIFT.jpg")
avg_ssim, scores = compute_stitched_ssim("156_SIFT.png", 5, seam_width=50, black_thresh=10)
print(f"Average SSIM for seams: {avg_ssim:.4f}")
print("Vertical seam scores:", scores["vertical"])
print("Horizontal seam scores:", scores["horizontal"])


156_KAZE.png
Average SSIM for seams: 0.9819
Vertical seam scores: [np.float64(0.8551943471596378), np.float64(0.7318238917075016), np.float64(0.4282228553023439), np.float64(0.571422808286549), np.float64(0.6818027853484027), np.float64(0.7211800908130588), np.float64(0.8253758982166604), np.float64(0.41656480801142814), np.float64(0.7118285981270328), np.float64(0.529849762386987), np.float64(0.844405486904106), np.float64(0.42596888608341615), np.float64(0.6014729104286264), np.float64(0.7724454512625942), np.float64(0.6680267321633925), np.float64(0.595736396198919), np.float64(0.707397988753125), np.float64(0.5167141214409228), np.float64(0.594049484135823), np.float64(0.8263853184985149)]
Horizontal seam scores: [np.float64(0.8019238928887318), np.float64(0.7686055484249492), np.float64(0.7247212140710834), np.float64(0.5306653487242875), np.float64(0.5993458576934653), np.float64(0.7112108511299987), np.float64(0.5760077777803255), np.float64(0.6460995098723982), np.float64(0.81

In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

def crop_black_borders(image, threshold=10):
    """
    Crop out surrounding black (or nearly black) borders from the image.
    A pixel is considered non-black if any channel exceeds `threshold`.
    """
    if image.ndim == 3:
        mask = image.max(axis=2) > threshold
    else:
        mask = image > threshold

    coords = np.argwhere(mask)
    if coords.size == 0:
        return image  # nothing but black
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0) + 1
    return image#[y0:y1, x0:x1]


def compute_scan_ssim(
    image_path: str,
    patch_h: int,
    patch_w: int,
    stride: int = 1,
    seam_width: int = 10,
    black_thresh: int = 10
):
    """
    Slide a window of size patch_h×patch_w across the image and compute
    SSIM between each window and its right and bottom neighbor windows.

    Returns:
      vert_map: 2D array of vertical-seam SSIM
      hori_map: 2D array of horizontal-seam SSIM
      mean_ssim: arithmetic mean of all valid seam scores
      rms_ssim: root-mean-square of all valid seam scores
    """
    # Load & crop
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        raise FileNotFoundError(f"Cannot open {image_path!r}")
    img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img = crop_black_borders(img, threshold=black_thresh)
    H, W = img.shape[:2]

    # Number of sliding windows
    n_rows = (H - patch_h) // stride + 1
    n_cols = (W - patch_w) // stride + 1

    # Allocate maps
    vert_map = np.full((n_rows, n_cols - 1), np.nan, dtype=np.float32)
    hori_map = np.full((n_rows - 1, n_cols), np.nan, dtype=np.float32)

    # Slide & compute SSIM
    for i in range(n_rows):
        y = i * stride
        for j in range(n_cols):
            x = j * stride
            patch = img[y : y + patch_h, x : x + patch_w]

            # Vertical seam (to right neighbor)
            if j + 1 < n_cols:
                nx = x + stride
                right = img[y : y + patch_h, nx : nx + patch_w]
                left_seam  = patch[:, -seam_width :]
                right_seam = right[:, : seam_width]
                lg = cv2.cvtColor(left_seam,  cv2.COLOR_RGB2GRAY)
                rg = cv2.cvtColor(right_seam, cv2.COLOR_RGB2GRAY)
                vert_map[i, j] = ssim(lg, rg)

            # Horizontal seam (to bottom neighbor)
            if i + 1 < n_rows:
                ny = y + stride
                bottom = img[ny : ny + patch_h, x : x + patch_w]
                top_seam    = patch[-seam_width :, :]
                bottom_seam = bottom[: seam_width, :]
                tg = cv2.cvtColor(top_seam,    cv2.COLOR_RGB2GRAY)
                bg = cv2.cvtColor(bottom_seam, cv2.COLOR_RGB2GRAY)
                hori_map[i, j] = ssim(tg, bg)

    # Flatten out all valid scores
    all_scores = np.concatenate([
        vert_map[~np.isnan(vert_map)],
        hori_map[~np.isnan(hori_map)]
    ])

    # Compute averages
    mean_ssim = np.mean(all_scores) if all_scores.size else np.nan
    rms_ssim  = np.sqrt(np.mean(all_scores**2)) if all_scores.size else np.nan

    return vert_map, hori_map, mean_ssim, rms_ssim


# Example usage
if __name__ == "__main__":
    V, H, mean_score, rms_score = compute_scan_ssim(
        "234_0000-0044_LightGlue_1.0.jpg",
        patch_h=200,
        patch_w=200,
        stride=50,
        seam_width=10,
        black_thresh=10
    )

    print("234_0000-0024_KAZE_0.7_0.25.jpg")
    print(f"Vertical map shape: {V.shape}")
    print(f"Horizontal map shape: {H.shape}")
    print(f"Mean SSIM across all seams: {mean_score:.4f}")
    print(f"RMS  SSIM across all seams: {rms_score:.4f}")

    V, H, mean_score, rms_score = compute_scan_ssim(
        "234_0000-0044_LightGlue_1.0_2.jpg",
        patch_h=200,
        patch_w=200,
        stride=50,
        seam_width=10,
        black_thresh=10
    )

    print("234_0000-0044_LightGlue_1.0_2.jpg")
    print(f"Vertical map shape: {V.shape}")
    print(f"Horizontal map shape: {H.shape}")
    print(f"Mean SSIM across all seams: {mean_score:.4f}")
    print(f"RMS  SSIM across all seams: {rms_score:.4f}")

    V, H, mean_score, rms_score = compute_scan_ssim(
        "194_0000-0044_LightGlue_0.25.jpg",
        patch_h=200,
        patch_w=200,
        stride=50,
        seam_width=10,
        black_thresh=10
    )

    print("194_0000-0044_LightGlue_0.25.jpg")
    print(f"Vertical map shape: {V.shape}")
    print(f"Horizontal map shape: {H.shape}")
    print(f"Mean SSIM across all seams: {mean_score:.4f}")
    print(f"RMS  SSIM across all seams: {rms_score:.4f}")

    V, H, mean_score, rms_score = compute_scan_ssim(
        "194_0000-0044_LightGlue_1.0.jpg",
        patch_h=200,
        patch_w=200,
        stride=50,
        seam_width=10,
        black_thresh=10
    )

    print("194_0000-0044_LightGlue_1.0.jpg")
    print(f"Vertical map shape: {V.shape}")
    print(f"Horizontal map shape: {H.shape}")
    print(f"Mean SSIM across all seams: {mean_score:.4f}")
    print(f"RMS  SSIM across all seams: {rms_score:.4f}")


FileNotFoundError: Cannot open '234_0000-0044_LightGlue_1.0.jpg'

In [ ]:
V, H, mean_score, rms_score = compute_scan_ssim(
    "sift_194_1.0.jpg",
    patch_h=200,
    patch_w=200,
    stride=50,
    seam_width=10,
    black_thresh=10
)

print(f"Vertical map shape: {V.shape}")
print(f"Horizontal map shape: {H.shape}")
print(f"Mean SSIM across all seams: {mean_score:.4f}")
print(f"RMS  SSIM across all seams: {rms_score:.4f}")

Vertical map shape: (139, 176)
Horizontal map shape: (138, 177)
Mean SSIM across all seams: 0.5485
RMS  SSIM across all seams: 0.5928


In [ ]:
print(V.min())

-0.022514218


In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

def crop_black_borders(image, threshold=10):
    if image.ndim == 3:
        mask = image.max(axis=2) > threshold
    else:
        mask = image > threshold
    coords = np.argwhere(mask)
    if coords.size == 0:
        return image
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0) + 1
    return image[y0:y1, x0:x1]

def compute_improved_scan_ssim(
    image_path: str,
    patch_h: int,
    patch_w: int,
    stride: int = 1,
    seam_width: int = 10,
    black_thresh: int = 10,
    α: float = 0.5,
    β: float = 0.4,
    γ: float = 0.1
):
    """
    Sliding‐window scan that computes a combined seam‐quality score:
      Q = α·(intensity SSIM)
        + β·(edge SSIM)
        - γ·(normalized gradient‐difference)
    Returns:
      vert_map, hori_map: arrays of Q‐scores
      mean_Q, rms_Q: overall statistics
    """
    # 1) load & crop
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        raise FileNotFoundError(f"Cannot open {image_path!r}")
    img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    # img = crop_black_borders(img, black_thresh)
    H, W = img.shape[:2]

    # 2) compute grid of sliding windows
    n_rows = (H - patch_h)//stride + 1
    n_cols = (W - patch_w)//stride + 1

    vert_map = np.full((n_rows, n_cols-1), np.nan, dtype=np.float32)
    hori_map = np.full((n_rows-1, n_cols), np.nan, dtype=np.float32)

    # helper: gradient magnitude
    def grad_mag(gray):
        gx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
        gy = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
        return cv2.magnitude(gx, gy)

    for i in range(n_rows):
        y = i*stride
        for j in range(n_cols):
            x = j*stride
            patch = img[y:y+patch_h, x:x+patch_w]

            # prepare raw gray and edge maps
            gray = cv2.cvtColor(patch, cv2.COLOR_RGB2GRAY)
            edges = cv2.Canny(gray, 100, 200)
            gmag = grad_mag(gray)

            # vertical seam
            if j+1 < n_cols:
                # neighbor patch
                nx = x + stride
                neigh = img[y:y+patch_h, nx:nx+patch_w]
                gray2 = cv2.cvtColor(neigh, cv2.COLOR_RGB2GRAY)
                edges2 = cv2.Canny(gray2, 100, 200)
                gmag2 = grad_mag(gray2)

                L = gray[:, -seam_width:]
                R = gray2[:, :seam_width]
                E1 = edges[:, -seam_width:]
                E2 = edges2[:, :seam_width]
                G1 = gmag[:, -seam_width:]
                G2 = gmag2[:, :seam_width]

                s_int = ssim(L, R)
                s_edge = ssim(E1, E2)
                # normalized gradient diff
                diff = np.abs(G1 - G2)
                g_diff = np.mean(diff) / (np.max([G1.max(), G2.max(), 1e-6]))
                vert_map[i,j] = α*s_int + β*s_edge - γ*g_diff

            # horizontal seam
            if i+1 < n_rows:
                ny = y + stride
                neigh = img[ny:ny+patch_h, x:x+patch_w]
                gray2 = cv2.cvtColor(neigh, cv2.COLOR_RGB2GRAY)
                edges2 = cv2.Canny(gray2, 100, 200)
                gmag2 = grad_mag(gray2)

                T = gray[-seam_width: , :]
                B = gray2[:seam_width, :]
                E1 = edges[-seam_width: , :]
                E2 = edges2[:seam_width, :]
                G1 = gmag[-seam_width: , :]
                G2 = gmag2[:seam_width, :]

                s_int = ssim(T, B)
                s_edge = ssim(E1, E2)
                diff = np.abs(G1 - G2)
                g_diff = np.mean(diff) / (np.max([G1.max(), G2.max(), 1e-6]))
                hori_map[i,j] = α*s_int + β*s_edge - γ*g_diff

    # flatten valid scores
    all_Q = np.concatenate([
        vert_map[~np.isnan(vert_map)],
        hori_map[~np.isnan(hori_map)]
    ])
    mean_Q = np.mean(all_Q) if all_Q.size else np.nan
    rms_Q  = np.sqrt(np.mean(all_Q**2)) if all_Q.size else np.nan

    return vert_map, hori_map, mean_Q, rms_Q

# # Example usage:
# if __name__ == "__main__":
#     V, H, μ, ρ = compute_improved_scan_ssim(
#         "sift_194_1.0.jpg",
#         patch_h=200, patch_w=200,
#         stride=50, seam_width=10,
#         black_thresh=10,
#         α=0.2, β=0.4, γ=0.4
#     )
#     print("sift_194_1.0.jpg")
#     print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "1_SIFT.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "1_SIFT_v2.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "1_SIFT_v3.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "1_SuperPoint.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "1_SuperPoint_v2.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


Mean Q-score: 0.4400, RMS Q-score: 0.4438
Mean Q-score: 0.2350, RMS Q-score: 0.2696
Mean Q-score: 0.2502, RMS Q-score: 0.2792
Mean Q-score: 0.3083, RMS Q-score: 0.3275
Mean Q-score: 0.3171, RMS Q-score: 0.3353


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "2_SIFT.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "2_SuperPoint.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

Mean Q-score: 0.2645, RMS Q-score: 0.2957
Mean Q-score: 0.2556, RMS Q-score: 0.2908


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "3_SIFT.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "3_SIFT_v2.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "3_SIFT_v3.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "3_SuperPoint.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "3_SuperPoint_v2.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


Mean Q-score: 0.3623, RMS Q-score: 0.3771
Mean Q-score: 0.2901, RMS Q-score: 0.3261
Mean Q-score: 0.3001, RMS Q-score: 0.3316
Mean Q-score: 0.3095, RMS Q-score: 0.3523
Mean Q-score: 0.3135, RMS Q-score: 0.3499


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "5_KAZE.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "5_SuperGlue.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "5_SIFT.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

Mean Q-score: 0.2834, RMS Q-score: 0.3086
Mean Q-score: 0.2331, RMS Q-score: 0.2670
Mean Q-score: 0.2709, RMS Q-score: 0.2991


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "6_SuperPoint.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "6_SuperPoint_original.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "6_SIFT.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "6_SIFT_original.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

Mean Q-score: 0.1850, RMS Q-score: 0.2477
Mean Q-score: 0.2314, RMS Q-score: 0.3135
Mean Q-score: 0.2138, RMS Q-score: 0.2652
Mean Q-score: 0.1745, RMS Q-score: 0.2593


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "7_KAZE.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "7_KAZE.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "7_SuperPoint.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


V, H, μ, ρ = compute_improved_scan_ssim(
    "7_SuperPoint.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "7_SIFT.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "7_SIFT.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

Mean Q-score: 0.4072, RMS Q-score: 0.4362
Mean Q-score: 0.3618, RMS Q-score: 0.3718
Mean Q-score: 0.3673, RMS Q-score: 0.4109
Mean Q-score: 0.3350, RMS Q-score: 0.3548
Mean Q-score: 0.2132, RMS Q-score: 0.2768
Mean Q-score: 0.2365, RMS Q-score: 0.2702


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "8_SuperPoint.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


V, H, μ, ρ = compute_improved_scan_ssim(
    "8_SuperPoint.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "8_SIFT.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "8_SIFT.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

Mean Q-score: 0.2067, RMS Q-score: 0.2825
Mean Q-score: 0.2466, RMS Q-score: 0.2900
Mean Q-score: 0.3111, RMS Q-score: 0.3513
Mean Q-score: 0.3087, RMS Q-score: 0.3401


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "9_KAZE.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "9_KAZE_v2.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "9_SuperPoint.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "9_SIFT.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

Mean Q-score: 0.4125, RMS Q-score: 0.4202
Mean Q-score: 0.2053, RMS Q-score: 0.2673
Mean Q-score: 0.3053, RMS Q-score: 0.3348
Mean Q-score: 0.3115, RMS Q-score: 0.3396


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "11_KAZE.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "11_KAZE.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "11_SuperPoint.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


V, H, μ, ρ = compute_improved_scan_ssim(
    "11_SuperPoint.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "11_SIFT.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "11_SIFT.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

Mean Q-score: 0.3398, RMS Q-score: 0.3701
Mean Q-score: 0.3419, RMS Q-score: 0.3659
Mean Q-score: 0.3555, RMS Q-score: 0.3950
Mean Q-score: 0.3595, RMS Q-score: 0.3944
Mean Q-score: 0.3313, RMS Q-score: 0.3601
Mean Q-score: 0.3384, RMS Q-score: 0.3626


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "12_KAZE.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "12_KAZE.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "12_SuperPoint.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


V, H, μ, ρ = compute_improved_scan_ssim(
    "12_SuperPoint.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "12_SIFT.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "12_SIFT.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

Mean Q-score: 0.5076, RMS Q-score: 0.5246
Mean Q-score: 0.2330, RMS Q-score: 0.2654
Mean Q-score: 0.2743, RMS Q-score: 0.3613
Mean Q-score: 0.2780, RMS Q-score: 0.3426
Mean Q-score: 0.2552, RMS Q-score: 0.3279
Mean Q-score: 0.2341, RMS Q-score: 0.2867


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "31_KAZE.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "31_KAZE.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "31_SuperPoint.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


V, H, μ, ρ = compute_improved_scan_ssim(
    "31_SuperPoint.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

# V, H, μ, ρ = compute_improved_scan_ssim(
#     "31_SIFT.jpg",
#     patch_h=200, patch_w=200,
#     stride=50, seam_width=10,
#     black_thresh=10,
#     α=0.2, β=0.4, γ=0.4
# )
# print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

# V, H, μ, ρ = compute_improved_scan_ssim(
#     "31_SIFT.png",
#     patch_h=200, patch_w=200,
#     stride=50, seam_width=10,
#     black_thresh=10,
#     α=0.2, β=0.4, γ=0.4
# )
# print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

Mean Q-score: 0.4367, RMS Q-score: 0.4525
Mean Q-score: 0.4316, RMS Q-score: 0.4427
Mean Q-score: 0.5215, RMS Q-score: 0.5285
Mean Q-score: 0.4752, RMS Q-score: 0.4789


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "33_KAZE.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "33_KAZE.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

# V, H, μ, ρ = compute_improved_scan_ssim(
#     "33_SuperPoint.jpg",
#     patch_h=200, patch_w=200,
#     stride=50, seam_width=10,
#     black_thresh=10,
#     α=0.2, β=0.4, γ=0.4
# )
# print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


# V, H, μ, ρ = compute_improved_scan_ssim(
#     "33_SuperPoint.png",
#     patch_h=200, patch_w=200,
#     stride=50, seam_width=10,
#     black_thresh=10,
#     α=0.2, β=0.4, γ=0.4
# )
# print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "33_SIFT.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "33_SIFT.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

Mean Q-score: 0.4329, RMS Q-score: 0.4546
Mean Q-score: 0.3999, RMS Q-score: 0.4062
Mean Q-score: 0.4819, RMS Q-score: 0.4942
Mean Q-score: 0.4354, RMS Q-score: 0.4388


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "36_KAZE.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "36_KAZE.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "36_SuperPoint.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


V, H, μ, ρ = compute_improved_scan_ssim(
    "36_SuperPoint.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "36_SIFT.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "36_SIFT.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

Mean Q-score: 0.4508, RMS Q-score: 0.4684
Mean Q-score: 0.3598, RMS Q-score: 0.3728
Mean Q-score: 0.3960, RMS Q-score: 0.4126
Mean Q-score: 0.3874, RMS Q-score: 0.3946
Mean Q-score: 0.4320, RMS Q-score: 0.4499
Mean Q-score: 0.4052, RMS Q-score: 0.4109


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "49_KAZE.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "49_KAZE.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "49_SuperPoint.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


V, H, μ, ρ = compute_improved_scan_ssim(
    "49_SuperPoint.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "49_SIFT.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "49_SIFT.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

Mean Q-score: 0.4268, RMS Q-score: 0.4461
Mean Q-score: 0.4231, RMS Q-score: 0.4361
Mean Q-score: 0.4930, RMS Q-score: 0.5084
Mean Q-score: 0.4569, RMS Q-score: 0.4631
Mean Q-score: 0.4560, RMS Q-score: 0.4712
Mean Q-score: 0.4251, RMS Q-score: 0.4311


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "53_KAZE.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "53_KAZE.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "53_SuperPoint.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


V, H, μ, ρ = compute_improved_scan_ssim(
    "53_SuperPoint.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "53_SIFT.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "53_SIFT.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

Mean Q-score: 0.3311, RMS Q-score: 0.3727
Mean Q-score: 0.3303, RMS Q-score: 0.3516
Mean Q-score: 0.3966, RMS Q-score: 0.4267
Mean Q-score: 0.3782, RMS Q-score: 0.3920
Mean Q-score: 0.3989, RMS Q-score: 0.4352
Mean Q-score: 0.3713, RMS Q-score: 0.3861


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "026_SIFT.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

Mean Q-score: 0.4811, RMS Q-score: 0.4938


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "051_KAZE.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "051_SIFT.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

Mean Q-score: 0.2927, RMS Q-score: 0.3287
Mean Q-score: 0.4401, RMS Q-score: 0.4458


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "156_KAZE.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "156_SIFT.png",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

Mean Q-score: 0.4208, RMS Q-score: 0.4356
Mean Q-score: 0.4669, RMS Q-score: 0.4856


In [ ]:
import cv2
import numpy as np
import os

def process_and_save_filters(image_path):
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not read the image from path: {image_path}")

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Canny filter
    canny = cv2.Canny(gray, 100, 200)
    canny_path = "canny_output.png"
    cv2.imwrite(canny_path, canny)

    # Apply Sobel filter (x-direction)
    sobelx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
    sobelx_display = cv2.convertScaleAbs(sobelx)  # for visualization
    sobelx_path = "sobelx_output.png"
    cv2.imwrite(sobelx_path, sobelx_display)

    # Apply Sobel filter (y-direction)
    sobely = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
    sobely_display = cv2.convertScaleAbs(sobely)  # for visualization
    sobely_path = "sobely_output.png"
    cv2.imwrite(sobely_path, sobely_display)

    # Compute gradient magnitude
    magnitude = cv2.magnitude(sobelx, sobely)
    magnitude_display = cv2.convertScaleAbs(magnitude)
    magnitude_path = "magnitude_output.png"
    cv2.imwrite(magnitude_path, magnitude_display)

    print("Saved images:")
    print(canny_path, sobelx_path, sobely_path, magnitude_path)

# Example usage
process_and_save_filters("image.jpg")


Saved images:
canny_output.png sobelx_output.png sobely_output.png magnitude_output.png


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "sift_234_1.0_offset_1_v2.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print("sift_234_1.0.png")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

sift_234_1.0.png
Mean Q-score: 0.4064, RMS Q-score: 0.4163


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "superpoint_194_0.25.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print("superpoint_194_0.25.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "superpoint_194_1.0.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print("superpoint_194_1.0.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

superpoint_194_0.25.jpg
Mean Q-score: 0.4106, RMS Q-score: 0.4229
superpoint_194_1.0.jpg
Mean Q-score: 0.4247, RMS Q-score: 0.4330


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "kaze_234_0.25.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print("kaze_234_0.25.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "kaze_234_0.75.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print("kaze_234_0.75.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "kaze_234_1.0.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print("kaze_234_1.0.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


V, H, μ, ρ = compute_improved_scan_ssim(
    "kaze_234_1.0_offset_1.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print("kaze_234_1.0_offset_1.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "superpoint_234_1.0.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print("superpoint_234_1.0.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "superpoint_234_1.0_offset_1.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.2, β=0.4, γ=0.4
)
print("superpoint_234_1.0_offset_1.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


kaze_234_0.25.jpg
Mean Q-score: 0.4014, RMS Q-score: 0.4159
kaze_234_0.75.jpg
Mean Q-score: 0.3690, RMS Q-score: 0.3903
kaze_234_1.0.jpg
Mean Q-score: 0.3997, RMS Q-score: 0.4151
kaze_234_1.0_offset_1.jpg
Mean Q-score: 0.3850, RMS Q-score: 0.4019
superpoint_234_1.0.jpg
Mean Q-score: 0.4325, RMS Q-score: 0.4426
superpoint_234_1.0_offset_1.jpg
Mean Q-score: 0.3690, RMS Q-score: 0.3875


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "234_0000-0024_KAZE_0.7_0.25.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.01, β=0.98, γ=0.01
)
print("234_0000-0024_KAZE_0.7_0.25.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "234_0000-0044_LightGlue_1.0_2.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.01, β=0.98, γ=0.01
)
print("234_0000-0044_LightGlue_1.0_2.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

# V, H, μ, ρ = compute_improved_scan_ssim(
#     "194_0000-0044_LightGlue_0.25.jpg",
#     patch_h=200, patch_w=200,
#     stride=50, seam_width=10,
#     black_thresh=10,
#     α=0.01, β=0.98, γ=0.01
# )
# print("194_0000-0044_LightGlue_0.25.jpg")
# print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

# V, H, μ, ρ = compute_improved_scan_ssim(
#     "194_0000-0044_LightGlue_1.0.jpg",
#     patch_h=200, patch_w=200,
#     stride=50, seam_width=10,
#     black_thresh=10,
#     α=0.01, β=0.98, γ=0.01
# )
# print("194_0000-0044_LightGlue_1.0.jpg")
# print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


234_0000-0024_KAZE_0.7_0.25.jpg
Mean Q-score: 0.9067, RMS Q-score: 0.9151
234_0000-0044_LightGlue_1.0_2.jpg
Mean Q-score: 0.9320, RMS Q-score: 0.9387


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "234_0000-0024_KAZE_0.7_0.25.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.01, β=0.01, γ=0.98
)
print("234_0000-0024_KAZE_0.7_0.25.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "234_0000-0044_LightGlue_1.0_2.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=10,
    black_thresh=10,
    α=0.01, β=0.01, γ=0.98
)
print("234_0000-0044_LightGlue_1.0_2.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

# V, H, μ, ρ = compute_improved_scan_ssim(
#     "194_0000-0044_LightGlue_0.25.jpg",
#     patch_h=200, patch_w=200,
#     stride=50, seam_width=10,
#     black_thresh=10,
#     α=0.01, β=0.98, γ=0.01
# )
# print("194_0000-0044_LightGlue_0.25.jpg")
# print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

# V, H, μ, ρ = compute_improved_scan_ssim(
#     "194_0000-0044_LightGlue_1.0.jpg",
#     patch_h=200, patch_w=200,
#     stride=50, seam_width=10,
#     black_thresh=10,
#     α=0.01, β=0.98, γ=0.01
# )
# print("194_0000-0044_LightGlue_1.0.jpg")
# print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


234_0000-0024_KAZE_0.7_0.25.jpg
Mean Q-score: -0.1283, RMS Q-score: 0.1445
234_0000-0044_LightGlue_1.0_2.jpg
Mean Q-score: -0.1416, RMS Q-score: 0.1557


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "234_0000-0024_KAZE_0.7_0.25.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=20,
    black_thresh=10,
    α=0.98, β=0.01, γ=0.01
)
print("234_0000-0024_KAZE_0.7_0.25.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "234_0000-0044_LightGlue_1.0_2.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=20,
    black_thresh=10,
    α=0.98, β=0.01, γ=0.01
)
print("234_0000-0044_LightGlue_1.0_2.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

# V, H, μ, ρ = compute_improved_scan_ssim(
#     "194_0000-0044_LightGlue_0.25.jpg",
#     patch_h=200, patch_w=200,
#     stride=50, seam_width=20,
#     black_thresh=10,
#     α=0.98, β=0.01, γ=0.01
# )
# print("194_0000-0044_LightGlue_0.25.jpg")
# print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

# V, H, μ, ρ = compute_improved_scan_ssim(
#     "194_0000-0044_LightGlue_1.0.jpg",
#     patch_h=200, patch_w=200,
#     stride=50, seam_width=20,
#     black_thresh=10,
#     α=0.98, β=0.01, γ=0.01
# )
# print("194_0000-0044_LightGlue_1.0.jpg")
# print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


234_0000-0024_KAZE_0.7_0.25.jpg
Mean Q-score: 0.5607, RMS Q-score: 0.6164
234_0000-0044_LightGlue_1.0_2.jpg
Mean Q-score: 0.5885, RMS Q-score: 0.6375


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "234_0000-0024_KAZE_0.7_0.25.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=20,
    black_thresh=10,
    α=0.01, β=0.98, γ=0.01
)
print("234_0000-0024_KAZE_0.7_0.25.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "234_0000-0044_LightGlue_1.0_2.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=20,
    black_thresh=10,
    α=0.01, β=0.98, γ=0.01
)
print("234_0000-0044_LightGlue_1.0_2.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "194_0000-0044_LightGlue_0.25.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=20,
    black_thresh=10,
    α=0.01, β=0.98, γ=0.01
)
print("194_0000-0044_LightGlue_0.25.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "194_0000-0044_LightGlue_1.0.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=20,
    black_thresh=10,
    α=0.01, β=0.98, γ=0.01
)
print("194_0000-0044_LightGlue_1.0.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


234_0000-0024_KAZE_0.7_0.25.jpg
Mean Q-score: 0.9059, RMS Q-score: 0.9138
234_0000-0044_LightGlue_1.0.jpg
Mean Q-score: 0.8410, RMS Q-score: 0.8580
194_0000-0044_LightGlue_0.25.jpg
Mean Q-score: 0.8969, RMS Q-score: 0.9068


In [ ]:
V, H, μ, ρ = compute_improved_scan_ssim(
    "234_0000-0024_KAZE_0.7_0.25.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=20,
    black_thresh=10,
    α=0.01, β=0.01, γ=0.98
)
print("234_0000-0024_KAZE_0.7_0.25.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "234_0000-0044_LightGlue_1.0.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=20,
    black_thresh=10,
    α=0.01, β=0.01, γ=0.98
)
print("234_0000-0044_LightGlue_1.0.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "194_0000-0044_LightGlue_0.25.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=20,
    black_thresh=10,
    α=0.01, β=0.01, γ=0.98
)
print("194_0000-0044_LightGlue_0.25.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")

V, H, μ, ρ = compute_improved_scan_ssim(
    "194_0000-0044_LightGlue_1.0.jpg",
    patch_h=200, patch_w=200,
    stride=50, seam_width=20,
    black_thresh=10,
    α=0.01, β=0.01, γ=0.98
)
print("194_0000-0044_LightGlue_1.0.jpg")
print(f"Mean Q-score: {μ:.4f}, RMS Q-score: {ρ:.4f}")


In [ ]:
import torch
import cv2
import numpy as np
from matplotlib import pyplot as plt
import kornia as K
from kornia.feature import SuperPoint

# 1. CONFIGURATION
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Choose which SuperPoint variant / pretrained weights you want.
# Kornia provides a small default checkpoint; you can download others from their repo.
# Here, “superpoint_v1” is their official pretrained model.
SP_CONFIG = {
    'descriptor_dim': 256,
    'nms_dist': 4,
    'keypoint_threshold': 0.005,
    'max_keypoints': 1024,
    'cuda': torch.cuda.is_available(),
}


def load_image_grayscale(path: str, target_size: tuple = None) -> torch.Tensor:
    """
    Reads an image from disk, converts to grayscale float32 tensor in [0,1],
    and (optionally) resizes it to `target_size` = (H, W).
    Returns a torch.Tensor of shape (1, 1, H, W).
    """
    img_bgr = cv2.imread(path, cv2.IMREAD_COLOR)
    if img_bgr is None:
        raise FileNotFoundError(f"Could not read image at {path}")
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    if target_size is not None:
        img_gray = cv2.resize(img_gray, (target_size[1], target_size[0]), interpolation=cv2.INTER_AREA)
    # normalize to [0,1]
    img_gray = img_gray.astype(np.float32) / 255.0
    # add batch & channel dims: (1,1,H,W)
    tensor = torch.from_numpy(img_gray)[None, None, :, :]
    return tensor.to(DEVICE)

# 2. INSTANTIATE SuperPoint
# Kornia’s SuperPoint constructor will download the default “superpoint_v1” checkpoint
sp = SuperPoint(
    descriptor_dim=SP_CONFIG['descriptor_dim'],
    nms_dist=SP_CONFIG['nms_dist'],
    keypoint_threshold=SP_CONFIG['keypoint_threshold'],
    max_keypoints=SP_CONFIG['max_keypoints'],
).to(DEVICE)
sp.eval()

# 3. RUN INFERENCE on a sample image
img_path = '0000.jpg'   # ◀– replace with your own image path
img_tensor = load_image_grayscale(img_path)  # shape = (1,1,H,W)

with torch.no_grad():
    # output is a dict with fields: 'keypoints', 'scores', 'descriptors', plus intermediate heatmaps
    #   keypoints:  (B, N, 2)   in pixel coords (x,y)
    #   descriptors: (B, 256, N)  per-keypoint descriptors (normalized)
    #   heatmap/descriptor_map:  for visualization if needed
    output = sp(img_tensor)

# Unpack output
keypoints   = output['keypoints'][0].cpu().numpy()     # shape = (N, 2)
scores      = output['scores'][0].cpu().numpy()        # shape = (N,)
descriptors = output['descriptors'][0].cpu().numpy()   # shape = (256, N)
# If you want the raw descriptor “map” (Hc×Wc×256), use output['descriptors_map']:
#   desc_map = output['descriptor_map'][0]  # (256, Hc, Wc) tensor

print(f"Found {keypoints.shape[0]} keypoints.")
print("Example keypoint #0 (x, y):", keypoints[0],  "— descriptor (first 8 dims):", descriptors[:8, 0])


# 4. OPTIONAL: Visualize keypoints and descriptor map on top of the image
import matplotlib.pyplot as plt

# A) show keypoints on the original image
orig = cv2.imread(img_path)
if orig is None:
    raise RuntimeError("Cannot load original image for visualization.")
orig_rgb = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

ax[0].imshow(orig_rgb)
ax[0].scatter(keypoints[:, 0], keypoints[:, 1], s=5, c='r', marker='o')
ax[0].set_title(f"Detected SuperPoint Keypoints (N={keypoints.shape[0]})")
ax[0].axis('off')

# B) visualize a single descriptor channel (e.g., channel #0) as a heatmap
#    First, upsample the descriptor_map to full image size (optional)
desc_map = output['descriptor_map'][0]  # shape = (256, Hc, Wc)
_, Hc, Wc = desc_map.shape
H_orig, W_orig = orig.shape[:2]
# pick channel 0
channel0 = desc_map[0]  # (Hc, Wc)
channel0_np = channel0.cpu().numpy()
heatmap = cv2.resize(channel0_np, (W_orig, H_orig), interpolation=cv2.INTER_CUBIC)
# normalize heatmap for display
heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min() + 1e-8)

ax[1].imshow(heatmap, cmap='inferno')
ax[1].set_title("SuperPoint Descriptor Channel #0 (upsampled)")
ax[1].axis('off')

plt.tight_layout()
plt.show()


ImportError: cannot import name 'SuperPoint' from 'kornia.feature' (/usr/local/lib/python3.11/dist-packages/kornia/feature/__init__.py)

In [ ]:
! pip install torch torchvision kornia opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 77.4 MB/s eta 0:00:00


In [ ]:
import os
import shutil

# Dictionary for filename mapping
name_map = {
    "0000.png": "0000.png",
    "0005.png": "0001.png",
    "0001.png": "0002.png",
    "0010.png": "0003.png",
    "0006.png": "0004.png",
    "0002.png": "0005.png",
    "0015.png": "0006.png",
    "0011.png": "0007.png",
    "0007.png": "0008.png",
    "0003.png": "0009.png",
    "0020.png": "0010.png",
    "0016.png": "0011.png",
    "0012.png": "0012.png",
    "0008.png": "0013.png",
    "0004.png": "0014.png",
    "0021.png": "0015.png",
    "0017.png": "0016.png",
    "0013.png": "0017.png",
    "0009.png": "0018.png",
    "0022.png": "0019.png",
    "0018.png": "0020.png",
    "0014.png": "0021.png",
    "0023.png": "0022.png",
    "0019.png": "0023.png",
    "0024.png": "0024.png"
}

def rename_and_copy_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            base_name = os.path.basename(filename)
            if base_name in name_map:
                new_name = name_map[base_name]
                src = os.path.join(input_folder, filename)
                dst = os.path.join(output_folder, new_name)
                shutil.copyfile(src, dst)
                print(f"Copied {filename} as {new_name}")
            else:
                print(f"Skipping {filename}: not in mapping")

# Example usage:
rename_and_copy_images("./12_patches", "./12_patches_diagonal")


Copied 0006.png as 0004.png
Copied 0011.png as 0007.png
Copied 0020.png as 0010.png
Copied 0024.png as 0024.png
Copied 0000.png as 0000.png
Copied 0007.png as 0008.png
Copied 0009.png as 0018.png
Copied 0022.png as 0019.png
Copied 0021.png as 0015.png
Copied 0003.png as 0009.png
Copied 0002.png as 0005.png
Copied 0023.png as 0022.png
Copied 0004.png as 0014.png
Copied 0012.png as 0012.png
Copied 0015.png as 0006.png
Copied 0017.png as 0016.png
Copied 0019.png as 0023.png
Copied 0001.png as 0002.png
Copied 0016.png as 0011.png
Copied 0005.png as 0001.png
Copied 0014.png as 0021.png
Copied 0013.png as 0017.png
Copied 0010.png as 0003.png
Copied 0018.png as 0020.png
Copied 0008.png as 0013.png


In [ ]:
import os
import shutil

# Dictionary for filename mapping
name_map = {
    "0021.jpg": "0000.jpg",
    "0031.jpg": "0001.jpg",
    "0022.jpg": "0002.jpg",
    "0041.jpg": "0003.jpg",
    "0032.jpg": "0004.jpg",
    "0023.jpg": "0005.jpg",
    "0051.jpg": "0006.jpg",
    "0042.jpg": "0007.jpg",
    "0033.jpg": "0008.jpg",
    "0024.jpg": "0009.jpg",
    "0061.jpg": "0010.jpg",
    "0052.jpg": "0011.jpg",
    "0043.jpg": "0012.jpg",
    "0034.jpg": "0013.jpg",
    "0025.jpg": "0014.jpg",
    "0062.jpg": "0015.jpg",
    "0053.jpg": "0016.jpg",
    "0044.jpg": "0017.jpg",
    "0035.jpg": "0018.jpg",
    "0063.jpg": "0019.jpg",
    "0054.jpg": "0020.jpg",
    "0045.jpg": "0021.jpg",
    "0064.jpg": "0022.jpg",
    "0055.jpg": "0023.jpg",
    "0065.jpg": "0024.jpg"
}

def rename_and_copy_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            base_name = os.path.basename(filename)
            if base_name in name_map:
                new_name = name_map[base_name]
                src = os.path.join(input_folder, filename)
                dst = os.path.join(output_folder, new_name)
                shutil.copyfile(src, dst)
                print(f"Copied {filename} as {new_name}")
            else:
                print(f"Skipping {filename}: not in mapping")

# Example usage:
rename_and_copy_images("./026-01-91", "./026-01-91_diagonal")


Copied 0054.jpg as 0020.jpg
Copied 0063.jpg as 0019.jpg
Copied 0064.jpg as 0022.jpg
Copied 0024.jpg as 0009.jpg
Copied 0061.jpg as 0010.jpg
Copied 0044.jpg as 0017.jpg
Copied 0031.jpg as 0001.jpg
Copied 0051.jpg as 0006.jpg
Copied 0022.jpg as 0002.jpg
Copied 0045.jpg as 0021.jpg
Copied 0041.jpg as 0003.jpg
Copied 0032.jpg as 0004.jpg
Copied 0042.jpg as 0007.jpg
Copied 0021.jpg as 0000.jpg
Copied 0025.jpg as 0014.jpg
Copied 0065.jpg as 0024.jpg
Copied 0023.jpg as 0005.jpg
Copied 0035.jpg as 0018.jpg
Copied 0053.jpg as 0016.jpg
Copied 0052.jpg as 0011.jpg
Copied 0043.jpg as 0012.jpg
Copied 0055.jpg as 0023.jpg
Copied 0034.jpg as 0013.jpg
Copied 0033.jpg as 0008.jpg
Copied 0062.jpg as 0015.jpg


In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim


def calculate_ssim(reference_path: str, target_path: str) -> float:
    """
    Calculate the Structural Similarity Index (SSIM) between a reference image and a target image.
    If the target image shape does not match the reference image, it will be resized to match.

    Parameters:
    ----------
    reference_path : str
        File path to the reference image.
    target_path : str
        File path to the target image.

    Returns:
    -------
    float
        SSIM score between the two images (value between -1 and 1).

    Example:
    -------
    >>> score = calculate_ssim('ref.png', 'tgt.jpg')
    >>> print(f"SSIM: {score:.4f}")
    """
    # Load images
    ref = cv2.imread(reference_path)
    tgt = cv2.imread(target_path)

    if ref is None:
        raise FileNotFoundError(f"Reference image not found at {reference_path}")
    if tgt is None:
        raise FileNotFoundError(f"Target image not found at {target_path}")

    # Resize target if shapes differ
    if ref.shape != tgt.shape:
        tgt = cv2.resize(tgt, (ref.shape[1], ref.shape[0]), interpolation=cv2.INTER_AREA)

    # Convert to grayscale if colored
    if len(ref.shape) == 3 and ref.shape[2] == 3:
        ref_gray = cv2.cvtColor(ref, cv2.COLOR_BGR2GRAY)
        tgt_gray = cv2.cvtColor(tgt, cv2.COLOR_BGR2GRAY)
    else:
        ref_gray = ref
        tgt_gray = tgt

    # Compute SSIM
    score, _ = ssim(ref_gray, tgt_gray, full=True)
    return score


In [ ]:
score = calculate_ssim('9.png', '9_KAZE.png')
print(f"SSIM score: {score}")
score = calculate_ssim('9.png', '9_KAZE_v2.png')
print(f"SSIM score: {score}")
score = calculate_ssim('9.png', '9_SIFT.png')
print(f"SSIM score: {score}")
score = calculate_ssim('9.png', '9_SuperPoint.png')
print(f"SSIM score: {score}")
# score = calculate_ssim('3.png', '3_SuperPoint_v2.png')
# print(f"SSIM score: {score}")

SSIM score: 0.1447792940108486
SSIM score: 0.11664658823254925
SSIM score: 0.1485103037220247
SSIM score: 0.15302723043073463


In [ ]:
score = calculate_ssim('9.png', '0000-0024_KAZE_0.7_1.0.jpg')
print(f"SSIM score: {score}")
score = calculate_ssim('9.png', 'SuperPoint_0000-0024_LightGlue_1.0.jpg')
print(f"SSIM score: {score}")
score = calculate_ssim('9.png', 'SIFT_0000-0024_LightGlue_1.0.jpg')
print(f"SSIM score: {score}")
# score = calculate_ssim('9.png', '9_SuperPoint.png')
# print(f"SSIM score: {score}")
# score = calculate_ssim('3.png', '3_SuperPoint_v2.png')
# print(f"SSIM score: {score}")

SSIM score: 0.04215802917791147
SSIM score: 0.14407228558749904
SSIM score: 0.13803512159877762


In [ ]:
import lpips
import torch
from PIL import Image
from torchvision import transforms

# 1) Pick device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# 2) Initialize LPIPS model on device
loss_fn = lpips.LPIPS(net='alex').to(device)
loss_fn.eval()  # we only need inference

# 3) Prepare transform once
to_tensor = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

@torch.no_grad()  # disable gradient tracking for speed & memory savings
def compute_lpips(path_orig, path_recon):
    # Load original to get target size
    orig = Image.open(path_orig).convert('RGB')
    w, h = orig.size

    # Open & resize in one go
    recon = Image.open(path_recon).convert('RGB').resize((w, h), Image.BICUBIC)

    # Tensorize, normalize, add batch dim, move to device
    t0 = to_tensor(orig).unsqueeze(0).to(device)
    t1 = to_tensor(recon).unsqueeze(0).to(device)

    # (Optional) use half precision on GPU
    if device.type == 'cuda':
        loss_fn.half()
        t0 = t0.half()
        t1 = t1.half()

    # Compute
    dist = loss_fn(t0, t1)
    return dist.item()


Using device: cpu
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 237MB/s]


Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth


In [ ]:
orig_path = '1.png'
recon_path = '1_SIFT.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '1.png'
recon_path = '1_SIFT_v2.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '1.png'
recon_path = '1_SIFT_v3.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

LPIPS distance: 0.7701
LPIPS distance: 0.7864
LPIPS distance: 0.7764


In [ ]:
orig_path = '1.png'
recon_path = '1_SuperPoint.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '1.png'
recon_path = '1_SuperPoint_v2.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

LPIPS distance: 0.7551
LPIPS distance: 0.7497


In [ ]:
torch.cuda.empty_cache()

In [ ]:
orig_path = '2.png'
recon_path = '2_SuperPoint.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '2.png'
recon_path = '2_SIFT.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

LPIPS distance: 0.7727
LPIPS distance: 0.7625


In [ ]:
orig_path = '3.png'
recon_path = '3_SIFT.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '3.png'
recon_path = '3_SIFT_v2.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '3.png'
recon_path = '3_SIFT_v3.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

LPIPS distance: 0.6012
LPIPS distance: 0.6142
LPIPS distance: 0.5993


In [ ]:
orig_path = '3.png'
recon_path = '3_SuperPoint.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '3.png'
recon_path = '3_SuperPoint_v2.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

LPIPS distance: 0.6347
LPIPS distance: 0.6289


In [ ]:
orig_path = '9.png'
recon_path = '0000-0024_KAZE_0.7_1.0.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '9.png'
recon_path = 'SIFT_0000-0024_LightGlue_1.0.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '9.png'
recon_path = 'SuperPoint_0000-0024_LightGlue_1.0.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

LPIPS distance: 1.0196
LPIPS distance: 0.8101
LPIPS distance: 0.7954


In [ ]:
orig_path = '5.png'
recon_path = '5_KAZE.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '5.png'
recon_path = '5_SuperGlue.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '5.png'
recon_path = '5_SIFT.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

LPIPS distance: 0.5839
LPIPS distance: 0.5651
LPIPS distance: 0.5684


In [ ]:
orig_path = '5.png'
recon_path = '0000-0024_KAZE_0.7_1.0.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '5.png'
recon_path = '0000-0024_LightGlue_1.0.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '5.png'
recon_path = '0000-0024_SIFT_1.0.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

LPIPS distance: 0.8134
LPIPS distance: 0.7586
LPIPS distance: 0.7804


In [ ]:
orig_path = '6.png'
recon_path = '6_SuperPoint.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '6.png'
recon_path = '6_SIFT.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

LPIPS distance: 0.6223
LPIPS distance: 0.6054


In [ ]:
orig_path = '6.png'
recon_path = '6_SuperPoint_original.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '6.png'
recon_path = '6_SIFT_original.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

LPIPS distance: 0.6503
LPIPS distance: 0.6420


In [ ]:
orig_path = '6.png'
recon_path = '6_SuperPoint.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '6.png'
recon_path = '6_SuperPoint.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '6.png'
recon_path = '6_SIFT.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '6.png'
recon_path = '6_SIFT.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

LPIPS distance: nan
LPIPS distance: 0.7871
LPIPS distance: 0.8301
LPIPS distance: 0.7910


In [ ]:
orig_path = '7.png'
recon_path = '7_KAZE.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '7.png'
recon_path = '7_KAZE.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')


orig_path = '7.png'
recon_path = '7_SuperPoint.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '7.png'
recon_path = '7_SuperPoint.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '7.png'
recon_path = '7_SIFT.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '7.png'
recon_path = '7_SIFT.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

LPIPS distance: 0.6610
LPIPS distance: 0.5902
LPIPS distance: 0.6558
LPIPS distance: 0.6023
LPIPS distance: 0.6298
LPIPS distance: 0.5872


In [ ]:
orig_path = '8.png'
recon_path = '8_SuperPoint.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '8.png'
recon_path = '8_SIFT.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

LPIPS distance: 0.8054
LPIPS distance: 0.7999


In [ ]:
orig_path = '8.png'
recon_path = '8_SuperPoint.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '8.png'
recon_path = '8_SIFT.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

LPIPS distance: 0.7590
LPIPS distance: 0.7720


In [ ]:
orig_path = '11.png'
recon_path = '11_KAZE.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '11.png'
recon_path = '11_KAZE.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')


orig_path = '11.png'
recon_path = '11_SuperPoint.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '11.png'
recon_path = '11_SuperPoint.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '11.png'
recon_path = '11_SIFT.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '11.png'
recon_path = '11_SIFT.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

LPIPS distance: 0.7766
LPIPS distance: 0.7316
LPIPS distance: 0.8027
LPIPS distance: 0.7986
LPIPS distance: 0.7625
LPIPS distance: 0.7179


In [ ]:
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 933120000

orig_path = '12.png'
recon_path = '12_KAZE.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '12.png'
recon_path = '12_KAZE.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')


orig_path = '12.png'
recon_path = '12_SuperPoint.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '12.png'
recon_path = '12_SuperPoint.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '12.png'
recon_path = '12_SIFT.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '12.png'
recon_path = '12_SIFT.png'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

LPIPS distance: 0.9941
LPIPS distance: 0.7675
LPIPS distance: 0.8803
LPIPS distance: 0.8516
LPIPS distance: 0.8362
LPIPS distance: 0.7877


In [ ]:
 # orig_path = '9.png'
# recon_path = '9_KAZE.png'
# lpips_score = compute_lpips(orig_path, recon_path)
# print(f'LPIPS distance: {lpips_score:.4f}')

# orig_path = '9.png'
# recon_path = '9_KAZE_v2.png'
# lpips_score = compute_lpips(orig_path, recon_path)
# print(f'LPIPS distance: {lpips_score:.4f}')

# orig_path = '9.png'
# recon_path = '9_SuperPoint.png'
# lpips_score = compute_lpips(orig_path, recon_path)
# print(f'LPIPS distance: {lpips_score:.4f}')

# orig_path = '9.png'
# recon_path = '9_SIFT.png'
# lpips_score = compute_lpips(orig_path, recon_path)
# print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '9.png'
recon_path = '0000-0024_KAZE_0.7_1.0.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '9.png'
recon_path = 'SIFT_0000-0024_LightGlue_1.0.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

orig_path = '9.png'
recon_path = 'SuperPoint_0000-0024_LightGlue_1.0.jpg'
lpips_score = compute_lpips(orig_path, recon_path)
print(f'LPIPS distance: {lpips_score:.4f}')

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (157644841 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


LPIPS distance: 0.7217
LPIPS distance: 0.6221
LPIPS distance: 0.6074


In [ ]:
! pip install lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.2 MB/s eta 0:00:00


In [ ]:
from PIL import Image
from tqdm import tqdm

# Open the image
lista = ["234.jpg", "234_offset_1.jpg", "194.jpg"]

for path in tqdm(lista):
  image = Image.open(path)

  # Resize the image to a new size
  image = image.resize((1000, 1000))

  # Save the resized image
  image.save(f"resized_folder/{path}")

100%|██████████| 3/3 [00:00<00:00,  9.67it/s]
